In [1]:
import os
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

In [2]:


def read_data_from_csv(path):
    """Load datasets from CSV files.
    Args:
        path (str): Path to the CSV file.
    Returns:
        X (np.ndarray): Features of samples.
        y (np.ndarray): Labels of samples, only provided in the public datasets.
    """
    assert os.path.exists(path), f'File not found: {path}!'
    assert os.path.splitext(path)[
        -1] == '.csv', f'Unsupported file type {os.path.splitext(path)[-1]}!'

    data = pd.read_csv(path)
    column_list = data.columns.values.tolist()

    if 'Label' in column_list:
        # for the public dataset, label column is provided.
        column_list.remove('Label')
        X = data[column_list].values
        y = data['Label'].values
        return X, y
    else:
        # for the private dataset, label column is not provided.
        X = data[column_list].values
        return X
    

In [3]:
X_public, y_public = read_data_from_csv('assignment_8_public.csv')
print('Shape of X_public:', X_public.shape)  # n_sample, m_feature (6499, 22)
print('Shape of y_public:', y_public.shape)  # n_sample (6499,)

Shape of X_public: (6499, 22)
Shape of y_public: (6499,)


In [4]:
# Create a dictionary to map old column names to new column names
# cap-shape	cap-surface	cap-color	bruises	odor	gill-attachment	gill-spacing	gill-size	gill-color	stalk-shape	stalk-root	stalk-surface-above-ring	stalk-surface-below-ring	stalk-color-above-ring	stalk-color-below-ring	veil-type	veil-color	ring-number	ring-type	spore-print-color	population	habitat

new_column_names = {
    0: "cap-shape",
    1: "cap-surface",
    2: "cap-color",
    3: "bruises",
    4: "odor",
    5: "gill-attachment",
    6: "gill-spacing",
    7: "gill-size",
    8: "gill-color",
    9: "stalk-shape",
    10: "stalk-root",
    11: "stalk-surface-above-ring",
    12: "stalk-surface-below-ring",
    13: "stalk-color-above-ring",
    14: "stalk-color-below-ring",
    15: "veil-type",
    16: "veil-color",
    17: "ring-number",
    18: "ring-type",
    19: "spore-print-color",
    20: "population",
    21: "habitat"
}

# Use the rename() method to change the column names
df_x = pd.DataFrame(X_public).rename(columns=new_column_names)
df_y = pd.DataFrame(y_public).rename(columns={0: 'Label'})

df_public = pd.concat([df_x, df_y], axis=1).replace('?',np.nan)

df_public

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,Label
0,convex,fibrous,gray,no,foul,free,close,broad,chocolate,enlarging,...,pink,brown,partial,white,one,large,chocolate,solitary,woods,poisonous
1,knobbed,smooth,red,no,spicy,free,close,narrow,buff,tapering,...,pink,white,partial,white,one,evanescent,white,several,leaves,poisonous
2,convex,fibrous,white,no,none,free,crowded,broad,pink,tapering,...,white,white,partial,white,one,evanescent,brown,abundant,grasses,edible
3,flat,smooth,gray,no,none,free,crowded,broad,chocolate,tapering,...,white,white,partial,white,one,evanescent,brown,abundant,grasses,edible
4,convex,smooth,gray,no,none,free,crowded,broad,chocolate,tapering,...,white,white,partial,white,one,evanescent,brown,scattered,grasses,edible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6494,bell,smooth,gray,no,none,free,crowded,broad,gray,enlarging,...,white,white,partial,white,two,pendant,white,numerous,grasses,edible
6495,convex,scaly,gray,no,foul,free,close,broad,chocolate,enlarging,...,pink,brown,partial,white,one,large,chocolate,solitary,grasses,poisonous
6496,bell,scaly,yellow,bruises,almond,free,close,broad,black,enlarging,...,white,white,partial,white,one,pendant,black,scattered,meadows,edible
6497,convex,scaly,red,no,spicy,free,close,narrow,buff,tapering,...,white,white,partial,white,one,evanescent,white,several,paths,poisonous


In [31]:
df_public['stalk-root']

0       bulbous
1           NaN
2         equal
3         equal
4         equal
         ...   
6494        NaN
6495    bulbous
6496       club
6497        NaN
6498    bulbous
Name: stalk-root, Length: 6499, dtype: object

In [18]:
df_public.isna().sum()

cap-shape                      0
cap-surface                    0
cap-color                      0
bruises                        0
odor                           0
gill-attachment                0
gill-spacing                   0
gill-size                      0
gill-color                     0
stalk-shape                    0
stalk-root                  1974
stalk-surface-above-ring       0
stalk-surface-below-ring       0
stalk-color-above-ring         0
stalk-color-below-ring         0
veil-type                      0
veil-color                     0
ring-number                    0
ring-type                      0
spore-print-color              0
population                     0
habitat                        0
Label                          0
dtype: int64

In [5]:
pd.reset_option('^display.', silent=True)
df_public.value_counts(['stalk-root']).sort_index(ascending=True).to_frame()

,count
stalk-root,
bulbous,3029
club,452
equal,884
rooted,160


In [6]:
df_public.value_counts(['Label']).sort_index(ascending=True).to_frame()

,count
Label,
edible,3359
poisonous,3140


In [21]:
df_public.value_counts(['veil-type']).sort_index(ascending=True).to_frame()

,count
veil-type,
partial,6499


In [22]:
df_public.value_counts(['veil-color']).sort_index(ascending=True).to_frame()

,count
veil-color,
brown,84
orange,77
white,6332
yellow,6


In [ ]:
df_public.value_counts(['veil-color']).sort_index(ascending=True).to_frame()

In [23]:
df_public.value_counts(['ring-number']).sort_index(ascending=True).to_frame()

,count
ring-number,
none,28
one,5990
two,481


In [24]:
df_public.value_counts(['ring-type']).sort_index(ascending=True).to_frame()

,count
ring-type,
evanescent,2198
flaring,34
large,1061
none,28
pendant,3178


In [ ]:
# guessing stalk-root
'cap-shape', 
'odor', 
'gill-spacing', 
['stalk-surface-above-ring', 'stalk-surface-below-ring'],
['stalk-color-above-ring', 'stalk-color-below-ring']

In [25]:
df_public.value_counts(['cap-shape', 'stalk-root']).sort_index(ascending=True).to_frame()

count
cap-shape stalk-root       
bell      bulbous        35
          club          207
conical   bulbous         1
          club            2
convex    bulbous      1552
          club          222
          equal         431
          rooted         76
flat      bulbous      1426
          club           11
          equal         427
          rooted         84
knobbed   bulbous        15
          club           10
sunken    equal          26

In [26]:
df_public.value_counts(['odor', 'stalk-root']).sort_index(ascending=True).to_frame()

count
odor     stalk-root       
almond   bulbous        38
         club          210
         rooted         77
anise    bulbous        40
         club          208
         rooted         83
creosote bulbous       148
foul     bulbous      1292
musty    club           28
none     bulbous      1511
         club            6
         equal         676
pungent  equal         208

In [27]:
df_public.value_counts(['gill-spacing', 'stalk-root']).sort_index(ascending=True).to_frame()

count
gill-spacing stalk-root       
close        bulbous      2825
             club          446
             equal         283
             rooted        160
crowded      bulbous       204
             club            6
             equal         601

In [28]:
df_public.value_counts(['stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-root']).sort_index(ascending=True).to_frame()

count
stalk-surface-above-ring stalk-surface-below-ring stalk-root       
fibrous                  fibrous                  bulbous        63
                                                  equal         155
                         smooth                   bulbous        70
                                                  equal         148
scaly                    scaly                    bulbous        13
                                                  club            6
silky                    scaly                    club           28
                         silky                    bulbous      1061
smooth                   fibrous                  bulbous        72
                                                  equal         154
                         scaly                    rooted        160
                         smooth                   bulbous      1750
                                                  club          418
                                                  equal         427

In [29]:
df_public.value_counts(['stalk-color-above-ring', 'stalk-color-below-ring', 'stalk-root']).sort_index(ascending=True).to_frame()

count
stalk-color-above-ring stalk-color-below-ring stalk-root       
brown                  brown                  bulbous       128
                       buff                   bulbous       122
                       pink                   bulbous       117
buff                   brown                  bulbous       122
                       buff                   bulbous       112
                       pink                   bulbous       122
cinnamon               cinnamon               club           28
gray                   gray                   bulbous       158
                       pink                   bulbous       152
                       white                  bulbous       147
pink                   brown                  bulbous       114
                       buff                   bulbous       119
                       gray                   bulbous       150
                       pink                   bulbous       268
                       white                  bulbous       148
white                  brown                  bulbous        42
                       gray                   bulbous       153
                       pink                   bulbous       151
                       white                  bulbous       704
                                              club          418
                                              equal         884
                                              rooted        160
yellow                 yellow                 club            6

In [30]:
df_public.value_counts(['stalk-color-above-ring', 'stalk-root']).sort_index(ascending=True).to_frame()

count
stalk-color-above-ring stalk-root       
brown                  bulbous       367
buff                   bulbous       356
cinnamon               club           28
gray                   bulbous       457
pink                   bulbous       799
white                  bulbous      1050
                       club          418
                       equal         884
                       rooted        160
yellow                 club            6

By above analysis on stalk-root, we can fill the missing data with the rule:
1. odor(foul) --> bulbous
2. stalk-color-above-ring(brown | buff | gray | pink) --> bulbous
3. stalk-surface-above-ring(silky) & stalk-surface-below-ring(silky) --> bulbous
4. odor(pungent) --> equal



In [37]:
df_puNew = df_public

# Fill missing values based on mappings
df_puNew.loc[(df_puNew['odor'] == 'foul') & (df_puNew['stalk-root'].isna()), 'stalk-root'] = 'bulbous'
df_puNew.loc[(df_puNew['stalk-color-above-ring'].isin(['brown', 'buff', 'gray', 'pink'])) & (df_puNew['stalk-root'].isna()), 'stalk-root'] = 'bulbous'
df_puNew.loc[(df_puNew['stalk-surface-above-ring'] == 'silky') & (df_puNew['stalk-surface-below-ring'] == 'silky') & (df_puNew['stalk-root'].isna()), 'stalk-root'] = 'bulbous'
df_puNew.loc[(df_puNew['odor'] == 'pungent') & (df_puNew['stalk-root'].isna()), 'stalk-root'] = 'equal'


df_puNew.loc[(df_puNew['gill-spacing'] == 'close') & (df_puNew['stalk-root'].isna()), 'stalk-root'] = 'bulbous'
df_puNew.loc[(df_puNew['stalk-root'].isna()), 'stalk-root'] = 'equal'


In [39]:
df_puNew.isna().sum().sum()

0

In [41]:
df_puNew

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,Label
0,convex,fibrous,gray,no,foul,free,close,broad,chocolate,enlarging,...,pink,brown,partial,white,one,large,chocolate,solitary,woods,poisonous
1,knobbed,smooth,red,no,spicy,free,close,narrow,buff,tapering,...,pink,white,partial,white,one,evanescent,white,several,leaves,poisonous
2,convex,fibrous,white,no,none,free,crowded,broad,pink,tapering,...,white,white,partial,white,one,evanescent,brown,abundant,grasses,edible
3,flat,smooth,gray,no,none,free,crowded,broad,chocolate,tapering,...,white,white,partial,white,one,evanescent,brown,abundant,grasses,edible
4,convex,smooth,gray,no,none,free,crowded,broad,chocolate,tapering,...,white,white,partial,white,one,evanescent,brown,scattered,grasses,edible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6494,bell,smooth,gray,no,none,free,crowded,broad,gray,enlarging,...,white,white,partial,white,two,pendant,white,numerous,grasses,edible
6495,convex,scaly,gray,no,foul,free,close,broad,chocolate,enlarging,...,pink,brown,partial,white,one,large,chocolate,solitary,grasses,poisonous
6496,bell,scaly,yellow,bruises,almond,free,close,broad,black,enlarging,...,white,white,partial,white,one,pendant,black,scattered,meadows,edible
6497,convex,scaly,red,no,spicy,free,close,narrow,buff,tapering,...,white,white,partial,white,one,evanescent,white,several,paths,poisonous


In [ ]:
X_private = read_data_from_csv('assignment_8_private.csv')
print('Shape of X_private:', X_private.shape)  # k_sample, m_feature (1625, 22)
# remove and make your own predictions.
preds = [' '] * len(X_private)

'''
CODE HERE!
e.g.,
preds = [' '] * len(X_private)
'''

submission = pd.DataFrame({'Label': preds})
submission.to_csv('assignment_8.csv', index=True, index_label='Id')